In [1]:
import pandas as pd
import numpy as np
import unicodedata as ud
import matplotlib.pyplot as plt
import matplotlib
import math

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [3]:
dfpor = pd.read_csv('/home/sebastian/Facu/OD/guia03/alcohol/student-por.csv', low_memory = False)
dfmat = pd.read_csv('/home/sebastian/Facu/OD/guia03/alcohol/student-mat.csv', low_memory = False)

In [4]:
#df.info(memory_usage = 'deep')

In [5]:
dfpor.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


### Agrego el promedio de cada alumno

In [13]:
dfpor['promedio'] = (dfpor['G1'] + dfpor['G2'] + dfpor['G3']) / 3
dfmat['promedio'] = (dfmat['G1'] + dfmat['G2'] + dfmat['G3']) / 3

# Grupo de preguntas 1

### Los alumnos de Matemática que eligieron el colegio en función de la cercanía a su hogar tienen mejor promedio de calificación, que los alumnos de Portugués que eligieron el establecimiento con igual criterio

In [23]:
print('promedio matemática: %f' %(dfmat[dfmat['reason'] == 'home']['promedio'].mean()))
print('promedio portugués: %f' %(dfpor[dfpor['reason'] == 'home']['promedio'].mean()))


promedio matemática: 10.605505
promedio portugués: 11.874720


### Falso

### El consumo de alcohol durante los fines de semana tiene mas influencia en la salud de los alumnos que en el tiempo de estudio.

In [42]:
# Walc Weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
print(dfmat['Walc'].unique())
# health Current health status (numeric: from 1 - very bad to 5 - very good)
print(dfmat['health'].unique())
# studytime Weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
print(dfmat['studytime'].unique())

[1 3 2 4 5]
[3 5 1 2 4]
[2 3 1 4]


In [43]:
print(dfmat.groupby('Walc').agg({'health':['mean']}))
print(dfpor.groupby('Walc').agg({'health':['mean']}))
print(dfmat.groupby('Walc').agg({'studytime':['mean']}))
print(dfpor.groupby('Walc').agg({'studytime':['mean']}))

        health
          mean
Walc          
1     3.443709
2     3.576471
3     3.425000
4     3.882353
5     3.857143
        health
          mean
Walc          
1     3.368421
2     3.526667
3     3.550000
4     3.862069
5     3.822222
     studytime
          mean
Walc          
1     2.284768
2     1.976471
3     1.987500
4     1.647059
5     1.714286
     studytime
          mean
Walc          
1     2.121457
2     1.900000
3     1.933333
4     1.620690
5     1.577778


### Falso

### Los alumnos de la escuela Gabriel Pereira tienen mejor desempeño que los de la escuela Mousinho da Silveira, tanto para Matemática como para Portugués.

In [44]:
# sigo sin poder distinguir los alumnos de matemática de los de portugués
dfaux = dfmat.groupby('school').agg({'promedio':['mean']})
print('matemática:')
print(dfaux)

dfaux = dfpor.groupby('school').agg({'promedio':['mean']})
print('\nportugués:')
print(dfaux)

matemática:
         promedio
             mean
school           
GP      10.737345
MS      10.239130
portugués:
         promedio
             mean
school           
GP      12.235619
MS      10.482301


### Verdadero

### Existen mas alumnos de Portugués en la escuela Mousinho da Silveira que en la Gabriel Pereira.

In [49]:
print(dfpor.groupby('school').size())

school
GP    423
MS    226
dtype: int64


### Falso

# Grupo de preguntas 2

### La mayoría de las familias tienen hasta 3 miembros

In [51]:
print(dfmat['famsize'].unique())
print(dfpor['famsize'].unique())

['GT3' 'LE3']
['GT3' 'LE3']


In [57]:
print(dfmat.groupby('famsize').size() + dfpor.groupby('famsize').size())

famsize
GT3    738
LE3    306
dtype: int64


### Falso

### Los alumnos con ambos padres en la casa tienen mejor desempeño que los padres que tienen un padre trabajando.

In [74]:
# Ambos en casa puede significar que:
#   - Mjob == 'at_home' y Fjob == 'at_home'
dfmat['at_home'] = (dfmat['Mjob'] == 'at_home').astype(int) + (dfmat['Fjob'] == 'at_home').astype(int)
dfaux = dfmat.groupby('at_home').agg({'promedio':['mean']})
print('matemática:')
print(dfaux)

dfpor['at_home'] = (dfpor['Mjob'] == 'at_home').astype(int) + (dfpor['Fjob'] == 'at_home').astype(int)
dfaux = dfpor.groupby('at_home').agg({'promedio':['mean']})
print('\nportugués:')
print(dfaux)

matemática:
          promedio
              mean
at_home           
0        10.892673
1         9.348718
2        13.190476

portugués:
          promedio
              mean
at_home           
0        11.931293
1        10.572104
2        11.537037


#### Si fuera el caso: Verdadero (Pero otra pregunta, más abajo, es claramente verdadera)

In [ ]:
# Ambos en casa puede significar que:
#   - Mjob == 'at_home' y/o Fjob == 'at_home'

In [90]:
dfmat['at_home'] = (dfmat['Mjob'] == 'at_home') & (dfmat['Fjob'] == 'at_home')
dfaux = dfmat.groupby('at_home')
matsuma = dfaux.agg({'promedio':['sum']})
matsize = dfaux.agg({'promedio':['size']})

dfpor['at_home'] = (dfpor['Mjob'] == 'at_home') & (dfpor['Fjob'] == 'at_home')
dfaux = dfpor.groupby('at_home')
porsuma = dfaux.agg({'promedio':['sum']})
porsize = dfaux.agg({'promedio':['size']})

suma = (matsuma + porsuma).astype(float)
size = (matsize + porsize).astype(float)
print(suma)
print(size)
# Porque no puedo dividir?
# dfaux = suma / size

        promedio
             sum
at_home         
False    11463.0
True       300.0
        promedio
            size
at_home         
False     1019.0
True        25.0


#### Si fuera el caso: Verdadero (Pero otra pregunta, más abajo, es claramente verdadera)

In [91]:
# Ambos en casa puede significar que:
#   - Pstatus == 'T'  (living together)
mask = (dfmat['Pstatus'] == 'T')
dfaux = dfmat.loc[mask]
dfaux = dfaux['promedio'].mean()
print('matemática:')
print(dfaux)

mask = (dfpor['Pstatus'] == 'T')
dfaux = dfpor.loc[mask]
dfaux = dfaux['promedio'].mean()
print('portugués:')
print(dfaux)

matemática:
10.62523540489642
portugués:
11.636789689513765


#### A esta altura lo daría por falso, es muy confuso y el otro de más abajo es evidentemente verdadero

### El contar con apoyo escolar adicional influye positivamente en las calificaciones.

In [58]:
# schoolsup Extra educational support (binary: yes or no)
print(dfmat['schoolsup'].unique())

['yes' 'no']


In [60]:
dfaux = dfmat.groupby('schoolsup').agg({'promedio':['mean']})
print('matemática:')
print(dfaux)

dfaux = dfpor.groupby('schoolsup').agg({'promedio':['mean']})
print('\nportugués:')
print(dfaux)

matemática:
            promedio
                mean
schoolsup           
no         10.875000
yes         9.359477

portugués:
            promedio
                mean
schoolsup           
no         11.690763
yes        11.063725


### Falso

### El nivel de educación de los padres en áreas urbanas, es en promedio mas alto que el de los que residen en áreas rurales.

In [62]:
# address Student's home address type (binary: 'U' - urban or 'R' - rural)
print(dfmat['address'].unique())
# Fedu Father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education, or 4 - higher education)
print(dfmat['Fedu'].unique())

['U' 'R']
[4 1 2 3 0]


In [65]:
dfaux = dfmat.groupby('address').agg({'Fedu':['mean']})
print('matemática: padres: ')
print(dfaux)

dfaux = dfpor.groupby('address').agg({'Fedu':['mean']})
print('\nportugués: padres: ')
print(dfaux)

matemática: padres: 
             Fedu
             mean
address          
R        2.375000
U        2.563518

portugués: padres: 
             Fedu
             mean
address          
R        2.071066
U        2.409292


### Verdadero

### No tengo la afirmación de la siguiente celda

In [21]:
data_sa_filtrado = df[df['Mjob'] != 'other']
data_sa_filtrado = df[df['Fjob'] != 'other']

dfigual = data_sa_filtrado[data_sa_filtrado['Mjob'] == data_sa_filtrado['Fjob']]
dfigual['total'] = dfigual['G1'] + dfigual['G2'] + dfigual['G3'] 
dfigual['promedio'] = dfigual['total'] / 3


dfdistinto = data_sa_filtrado[data_sa_filtrado['Mjob'] != data_sa_filtrado['Fjob']]
dfdistinto['total'] = dfdistinto['G1'] + dfdistinto['G2'] + dfdistinto['G3'] 
dfdistinto['promedio'] = dfdistinto['total'] / 3

print(dfigual['promedio'].mean())
print(dfdistinto['promedio'].mean())

12.081699346405232
11.420370370370373


/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d